## Initialization  

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))
%load_ext autoreload
%autoreload 2

In [2]:
import os 
import sys
sys.path.insert(0, '../src')
# sys.path.insert(0, '/home/kbardool/kusanagi/AdaSparseChem/src') ; print(sys.path)
import time
import argparse
import yaml
import types, copy, pprint
from time import sleep
from datetime import datetime
import pandas as pd
import numpy  as np
## Set visible GPU device 
##----------------------------------------------
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
# os.environ["CUDA_VISIBLE_DEVICES"] = '2'
# from pynvml import *
from utils import (initialize, init_dataloaders, init_environment, init_wandb, training_initializations, model_initializations, 
                   check_for_resume_training, disp_dataloader_info, disp_info_1, warmup_phase, weight_policy_training, 
                   display_gpu_info, init_dataloaders_by_fold_id, print_separator, print_heading, 
                   timestring, print_loss, get_command_line_args, load_from_pickle) 

pp = pprint.PrettyPrinter(indent=4)
np.set_printoptions(edgeitems=3, infstr='inf', linewidth=150, nanstr='nan')
pd.options.display.width = 132
# torch.set_printoptions(precision=6, threshold=None, edgeitems=None, linewidth=132, profile=None, sci_mode=None)
os.environ["WANDB_NOTEBOOK_NAME"] = "Adashare_Train.ipynb"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [3]:
import scipy
print(scipy.__version__)

1.8.0


## Create Environment

### Parse Input Args  - Read YAML config file - wandb initialization

In [4]:
# input_args = " --config yamls/chembl_3task_train.yaml " \
#              " --resume " \
#              " --exp_id      330i85cg" \
#              " --exp_name    0308_1204" \
#              " --lambda_sparsity  0.01"\
#              " --lambda_sharing   0.01" 
#========================================================================
#              " --no_residual "
#              " --exp_name       0410_1934 " \
#              " --hidden_size   100 100 100 100 100 100" \
#              " --tail_hidden_size  100 " \
#              " --decay_lr_rate      0.75"  \
#              " --decay_lr_freq       20"  \

In [5]:
# synthetic_1task_config = "../yamls/chembl_synt_train_1task.yaml"
# synthetic_3task_config = "../yamls/chembl_synt_train_3task.yaml"
# synthetic_5task_config = "../yamls/chembl_synt_train_5task.yaml"
# synthetic_config_file  = "../yamls/chembl_synt_train.yaml"
# config_file      = "../yamls/chembl_mini_train.yaml"
config_file      = "../yamls/chembl_cb29_train_1task.yaml"
config_file      = "../yamls/chembl_cb29_train_10task.yaml"
batch_size=4098
# batch_size=2048
# RESUME_MODEL_CKPT = 'model_train_ep_25_seed_0088'

####   For Resume 

In [6]:
restart_input_args = f" --config  {config_file} " \
             " --exp_desc     weight 105 bch/ep policy 105 bch/ep " \
             " --warmup_epochs        10 "  \
             " --hidden_size         4000 4000 4000 4000"  \
             " --tail_hidden_size    4000 "  \
             " --first_dropout       0.80 "  \
             " --middle_dropout      0.80"  \
             " --last_dropout        0.80 "  \
             " --seed_idx              0 "  \
             f" --batch_size        {batch_size} "  \
             " --task_lr           0.001 "  \
             " --backbone_lr       0.001 "  \
             " --decay_lr_rate       0.3 "  \
             " --decay_lr_freq        10 "  \
             " --policy_lr         0.001 "  \
             " --lambda_sparsity    0.02 "  \
             " --lambda_sharing     0.01 "  \
             " --pytorch_threads       7 "  \
             " --cuda_devices          2"   \
             " --gpu_ids               0 "  \
             " --resume"                    \
             " --resume_path        ../../experiments/AdaSparseChem-cb29-10task/4000x6_0817_1521_lr0.001_do0.8/" \
             " --resume_ckpt        model_best_model" \
             " --resume_metrics     metrics_best.pickle" \
             " --exp_id             2h7wyaua" \
             " --exp_name           0817_1521" \
             " --folder_sfx         POLICY_1"

####  For Initiating 

In [7]:
input_args = f" --config  {config_file} " \
             " --exp_desc            10-task with policy training " \
             " --warmup_epochs         10 "  \
             " --hidden_size         4000 4000 4000 4000 4000 4000 "  \
             " --tail_hidden_size    4000 "  \
             " --first_dropout       0.80 "  \
             " --middle_dropout      0.80"  \
             " --last_dropout        0.80 "  \
             " --seed_idx              0 "  \
             f" --batch_size        {batch_size} "  \
             " --task_lr           0.001 "  \
             " --backbone_lr       0.001 "  \
             " --decay_lr_rate       0.3 "  \
             " --decay_lr_freq        10 "  \
             " --policy_lr         0.001 "  \
             " --lambda_sparsity    0.02 "  \
             " --lambda_sharing     0.01 "  \
             " --pytorch_threads       7 "  \
             " --cuda_devices          2"   \
             " --gpu_ids               0 "  


### Read Configuration File

In [8]:
ns = types.SimpleNamespace()
input_args = input_args.split() if input_args is not None else input_args
# input_args = restart_input_args.split() 
ns.args = get_command_line_args(input_args, display = True)
os.environ["CUDA_VISIBLE_DEVICES"]=ns.args.cuda_devices


  command line parms : 
------------------------
 config...................  ../yamls/chembl_cb29_train_10task.yaml
 project_name.............  None
 exp_id...................  30ugid3y
 exp_name.................  None
 folder_sfx...............  None
 exp_desc.................  10-task with policy training
 hidden_sizes.............  [4000, 4000, 4000, 4000, 4000, 4000]
 tail_hidden_size.........  [4000]
 warmup_epochs............  10
 training_epochs..........  None
 seed_idx.................  0
 batch_size...............  4098
 first_dropout............  0.8
 middle_dropout...........  0.8
 last_dropout.............  0.8
 backbone_lr..............  0.001
 task_lr..................  0.001
 policy_lr................  0.001
 decay_lr_rate............  0.3
 decay_lr_freq............  10.0
 lambda_sparsity..........  0.02
 lambda_sharing...........  0.01
 cuda_devices.............  2
 gpu_ids..................  [0]
 pytorch_threads..........  7
 skip_residual............  False
 skip_hi

In [9]:

# display_gpu_info()

In [10]:
opt = initialize(ns, build_folders = True)

##################################################
################### READ YAML ####################
##################################################
 Pytorch thread count: 20
 Set Pytorch thread count to : 7
 Pytorch thread count set to : 7


wandb: Currently logged in as: kbardool. Use `wandb login --relogin` to force relogin


 WandB Initialization -----------------------------------------------------------
 PROJECT NAME: AdaSparseChem-cb29-10Task
 RUN ID      : 30ugid3y 
 RUN NAME    : 0818_1529
 --------------------------------------------------------------------------------


 log_dir              create folder:  ../../experiments/AdaSparseChem-cb29-10task/4000x6_0818_1529_lr0.001_do0.8
 result_dir           folder exists:  ../../experiments/AdaSparseChem-cb29-10task/4000x6_0818_1529_lr0.001_do0.8
 checkpoint_dir       folder exists:  ../../experiments/AdaSparseChem-cb29-10task/4000x6_0818_1529_lr0.001_do0.8

------------------------------------------------------------------------------------------------------------------------
 experiment name       : 0818_1529 
 experiment id         : 30ugid3y 
 folder_name           : 4000x6_0818_1529_lr0.001_do0.8 
 experiment description: 10-task with policy training
 Random seeds          : [88, 45, 50, 100, 44, 48, 2048, 2222, 9999]
 Random  seed used     : 88 
 l

In [11]:
# ns.wandb_run.finish()

# ns.wandb_run.finish()

### Setup Dataloader

In [12]:
# dldrs = init_dataloaders(opt, verbose = False)
dldrs = init_dataloaders_by_fold_id(opt, verbose = False)
disp_dataloader_info(dldrs)
# folding  = np.load(os.path.join('/mnt/f/Chembl29', opt['dataload']['folding']))
# print(folding.shape)
# print(folding.max())

 Warmup folds    : [2, 3, 4]
 Weights folds   : [2, 3]
 Policy folds    : [4]
 Validation folds: [1]
---------------------------------------------------------------------------
Load label/Y file for task 1 - task group chembl_29_Y_tg_0_cols_472.npy
--------------------------------------------------------------------------- 

 Number of non-zero features in ecfp[0]:79

 Task 1 label file: 
    Total > +1  Labels :          0 
    Total   +1  Labels :      81937 
    Total   -1  Labels :     188511 
    Total < -1  Labels :          0 
    Total != 0  Labels :     270448

  Task 1 files pre-filtering : 
--------------------------------
X file : # Samples :  423736     # Features per Sample: 32000   
Y file : # Samples :  423736     # Labels per Sample  : 472  Y rows with populated labels: 32866  non zero cols: 81937

  Task 1 files post-filtering : 
---------------------------------
X file : # Samples :  254529     # Features per Sample: 32000 
Y file : # Samples :  254529     # Labels p


  Task 9 files pre-filtering : 
--------------------------------
X file : # Samples :  423736     # Features per Sample: 32000   
Y file : # Samples :  423736     # Labels per Sample  : 344  Y rows with populated labels: 35996  non zero cols: 110249

  Task 9 files post-filtering : 
---------------------------------
X file : # Samples :  254529     # Features per Sample: 32000 
Y file : # Samples :  254529     # Labels per Sample  : 344  Y rows with populated labels: 20684  non zero cols: 63517

Using 226 of 344 classification tasks for calculating aggregated metrics (AUCROC, F1_max, etc).
------------------------------------------------------------------------------
Load label/Y file for task 10 - task group chembl_29_Y_tg_1031_cols_72.npy
------------------------------------------------------------------------------ 

 Number of non-zero features in ecfp[0]:79

 Task 10 label file: 
    Total > +1  Labels :          0 
    Total   +1  Labels :      18631 
    Total   -1  Labels :   


  Task 7 files post-filtering : 
---------------------------------
X file : # Samples :  168649     # Features per Sample: 32000 
Y file : # Samples :  168649     # Labels per Sample  : 224  Y rows with populated labels: 4476  non zero cols: 14840

Using 109 of 224 classification tasks for calculating aggregated metrics (AUCROC, F1_max, etc).
------------------------------------------------------------------------------
Load label/Y file for task 8 - task group chembl_29_Y_tg_1005_cols_148.npy
------------------------------------------------------------------------------ 

 Number of non-zero features in ecfp[0]:79

 Task 8 label file: 
    Total > +1  Labels :          0 
    Total   +1  Labels :      45065 
    Total   -1  Labels :     104361 
    Total < -1  Labels :          0 
    Total != 0  Labels :     149426

  Task 8 files pre-filtering : 
--------------------------------
X file : # Samples :  423736     # Features per Sample: 32000   
Y file : # Samples :  423736     # Labe


  Task 7 files pre-filtering : 
--------------------------------
X file : # Samples :  423736     # Features per Sample: 32000   
Y file : # Samples :  423736     # Labels per Sample  : 224  Y rows with populated labels: 11789  non zero cols: 38227

  Task 7 files post-filtering : 
---------------------------------
X file : # Samples :  85880     # Features per Sample: 32000 
Y file : # Samples :  85880     # Labels per Sample  : 224  Y rows with populated labels: 2614  non zero cols: 7798

Using 109 of 224 classification tasks for calculating aggregated metrics (AUCROC, F1_max, etc).
------------------------------------------------------------------------------
Load label/Y file for task 8 - task group chembl_29_Y_tg_1005_cols_148.npy
------------------------------------------------------------------------------ 

 Number of non-zero features in ecfp[0]:79

 Task 8 label file: 
    Total > +1  Labels :          0 
    Total   +1  Labels :      45065 
    Total   -1  Labels :     1043


  Task 5 files pre-filtering : 
--------------------------------
X file : # Samples :  423736     # Features per Sample: 32000   
Y file : # Samples :  423736     # Labels per Sample  : 620  Y rows with populated labels: 51001  non zero cols: 142158

  Task 5 files post-filtering : 
---------------------------------
X file : # Samples :  86274     # Features per Sample: 32000 
Y file : # Samples :  86274     # Labels per Sample  : 620  Y rows with populated labels: 10004  non zero cols: 27223

Using 389 of 620 classification tasks for calculating aggregated metrics (AUCROC, F1_max, etc).
-----------------------------------------------------------------------------
Load label/Y file for task 6 - task group chembl_29_Y_tg_643_cols_184.npy
----------------------------------------------------------------------------- 

 Number of non-zero features in ecfp[0]:79

 Task 6 label file: 
    Total > +1  Labels :          0 
    Total   +1  Labels :      41813 
    Total   -1  Labels :      698

In [13]:
dldrs.valset.class_output_size

[472, 624, 688, 192, 620, 184, 224, 148, 344, 72]

## Setup Model  

In [14]:

environ = init_environment(ns, opt, is_train = True, display_cfg = True)


##################################################
############# CREATE THE ENVIRONMENT #############
##################################################
 device is  cuda:0
--------------------------------------------------
 SparseChem_Backbone  Ver: 1.0 Init() Start 
-------------------------------------------------- 

 layer config        : [1, 1, 1, 1, 1, 1] 
 skip residual layers: False   skip hidden layers  : False
 SparseChem_BackBone() Input Layer  - Input: 32000  output: 4000  non-linearity:<class 'torch.nn.modules.activation.ReLU'>
 Hidden layer 0 - Input: 4000   output:4000
    _make_layer() using block: <class 'models.sparsechem_backbone.SparseChemBlock'>
           input_size: 4000 output_sz: 4000  non_linearity: ReLU() dropout: 0.8 bias: True
           SparseChemBlock.init(): input_size: 4000 output_sz: 4000   non_linearity: ReLU() dropout: 0.8 bias: True
 Hidden layer 1 - Input: 4000   output:4000
    _make_layer() using block: <class 'models.sparsechem_backbone.SparseChe

In [15]:
# environ.optimizers['weights'].param_groups[0]
# print(environ.print_configuration())
# print(environ.networks['mtl-net']._arch_parameters)
# pp.pprint(environ.networks['mtl-net'].__dict__)

## Initiate / Resume Training Prep

In [16]:
check_for_resume_training(ns, opt, environ, epoch = 0 , iter = 0)

opt['train']['which_iter'] :  warmup
##################################################
######## Initiate Training from scratch  #########
##################################################


In [17]:
# loaded_metrics
# ns.val_metrics
# ns.best_roc_auc

In [18]:
# a = environ.load_checkpoint('warmup_ep_40_seed_0088', path = '../experiments/AdaSparseChem/50x6_0304_1549_plr0.01_sp0.0001_sh0.01/')
# current_iter = environ.load_checkpoint('latest_weights_policy')
# priRant('Evaluating the snapshot saved at %d iter' % current_iter)

## Warmup Training

### Warmup Training Preparation

In [19]:
model_initializations(ns, opt, environ, phase = 'update_weights', policy_learning = False)

 Model optimizers defined . . . policy_learning: False
 Model schedulers defined . . . policy_learning: False
 Metrics CSV file header written . . . 
 Model initializations complete . . . 


In [20]:
# print(hasattr(environ, 'policy%d' % (1)))
# print(getattr(environ, "policy1"))
# print(environ.networks['mtl-net'].policys)

In [21]:
# training_initializations(ns, opt, environ, dldrs, warmup_iterations = 200, weight_iterations = 200, policy_iterations = 200, eval_iterations = 50, warmup = True)
training_initializations(ns, opt, environ, dldrs, warmup = True)
# print('-'*80)
# disp_info_1(ns, opt, environ)
print('-'*80)
print(environ.disp_for_excel())

environ.display_trained_policy(ns.current_epoch,out=sys.stdout)
environ.display_trained_logits(ns.current_epoch,out=sys.stdout)

 training preparation: - check for CUDA - cuda available as device id: [0]
sparsechem_env.cuda()
 training preparation: - set print_freq to                                 : 1989 
 training preparation: - set number of batches per warmup training epoch to: 1989
 training preparation: - set number of batches per weight training epoch to: 1318
 training preparation: - set number of batches per policy training epoch to: 671
 training preparation: - set number of batches per validation to           : 675
 training preparation complete . . .
--------------------------------------------------------------------------------

    folder: 4000x6_0818_1529_lr0.001_do0.8
    layers: 6 [4000, 4000, 4000, 4000, 4000, 4000] 
    
    first dropout          : 0.8
    middle dropout         : 0.8
    last dropout           : 0.8
    diff_sparsity_weights  : False
    skip_layer             : 0
    is_curriculum          : False
    curriculum_speed       : 3
    
    task_lr                : 0.001
    

In [22]:
print(ns.eval_iters )
print(ns.trn_iters_warmup)
print(ns.trn_iters_weights)
print(ns.trn_iters_policy)

# ns.check_for_improvment_wait = 0
# ns.current_epoch =0 
# ns.write_checkpoint = False
print_heading(f" Last Epoch: {ns.current_epoch}   # of warm-up epochs to do:  {ns.warmup_epochs} - Run epochs {ns.current_epoch+1} to {ns.current_epoch + ns.warmup_epochs}", verbose = True)

675
1989
1318
671
------------------------------------------------------------------------
 Last Epoch: 0   # of warm-up epochs to do:  10 - Run epochs 1 to 10
------------------------------------------------------------------------ 



In [24]:

warmup_phase(ns,opt, environ, dldrs, epochs = 7, verbose = False, disable_tqdm = False)


-----------------------------------------------------------------------
 Last Epoch: 3   # of warm-up epochs to do:  7 - Run epochs 4 to 10
----------------------------------------------------------------------- 

 Ep  | Trunk LR  Heads LR  Polcy LR  Gmbl Tmp |  trn tsk    trn spar    trn shar   trn ttl |   logloss   bceloss  avg prec    aucroc     aucpr    f1_max |  val tsk    val spar    val shar     total | time |
   4 | 1.00e-03  1.00e-03  1.00e-03  4.00e+00 |   2.5792   2.133e-03   4.256e-05    2.5814 |   0.00000   0.49965   0.62771   0.71521   0.61453   0.67172 |   2.4291   1.184e-03   2.363e-05    2.4303 |389.0 |
Previous best_epoch:     3   best iter:  5967   best_accuracy: 0.62762    best ROC auc: 0.71362
Previous best_epoch:     4   best iter:  7956   best_accuracy: 0.62771    best ROC auc: 0.71521
 save best metrics to     :  metrics_best.pickle
 save  best checkpoint to :  model_best
   5 | 1.00e-03  1.00e-03  1.00e-03  4.00e+00 |   2.8063   2.133e-03   4.256e-05    2.8085 

In [25]:
pp.pprint(environ.val_metrics['total'])
print(environ.val_metrics['total']['task'] + environ.val_metrics['total']['policy'])

{   'policy': 0.0012077480350853875,
    'task': 2.261661641804446,
    'total': 2.2628693898395316,
    'total_mean': nan}
2.2628693898395316


In [27]:
print( f" Backbone Initial LR            :      {environ.opt['train']['backbone_lr']:4f}      Current LR : {environ.optimizers['weights'].param_groups[0]['lr']} \n"
       f" Tasks    Initial LR            :      {environ.opt['train']['task_lr']:4f}      Current LR : {environ.optimizers['weights'].param_groups[1]['lr']}    \n"
       f" Policy   Initial LR            :      {environ.opt['train']['policy_lr']:4f}      Current LR : {environ.optimizers['alphas'].param_groups[0]['lr']}  \n")

 Backbone Initial LR            :      0.001000      Current LR : 0.001 
 Tasks    Initial LR            :      0.001000      Current LR : 0.001    
 Policy   Initial LR            :      0.001000      Current LR : 0.001  



In [26]:
# ns.wandb_run.finish()
# ns.wandb_run.finish()

###  Aggregate Results

In [29]:
pp.pprint(environ.val_metrics['sparsity']['total'])
pp.pprint(environ.val_metrics['sharing']['total'])
pp.pprint(environ.val_metrics['sharing']['total'] +environ.val_metrics['sparsity']['total'])
pp.pprint(environ.val_metrics['task'])
pp.pprint(environ.val_metrics['total'])
pp.pprint(environ.val_metrics['epoch'])

0.0011841206578537822
2.362737723160535e-05
0.0012077480350853875
{   'task1': 0.22806914488911734,
    'task10': 0.07729441115807154,
    'task2': 0.2615806116049444,
    'task3': 0.6873060537261522,
    'task4': 0.1177855923062318,
    'task5': 0.352459824837004,
    'task6': 0.08289676805336933,
    'task7': 0.1123096017483592,
    'task8': 0.17276770787165321,
    'task9': 0.2891627654132546,
    'total': 2.381632481608156}
{   'policy': 0.0012077480350853875,
    'task': 2.381632481608156,
    'total': 2.3828402296432416,
    'total_mean': nan}
5


In [26]:
from utils             import censored_mse_loss, censored_mae_loss, aggregate_results
task_key = 'task2'
print(environ.val_data[task_key]['yc_aggr_weights'].sum())
print(environ.val_data[task_key]['yc_aggr_weights'])
print(environ.val_metrics[task_key]['classification'])
# print(environ.val_metrics[task_key]['classification'].sum())
print(environ.val_metrics[task_key]['classification_agg'])
# print(environ.val_data[task_key]['yc_aggr_weights'])
# print((environ.batch_data[task_key]['yc_aggr_weights']==environ.val_data[task_key]['yc_aggr_weights']).all())


tmp = aggregate_results(environ.val_metrics[task_key]["classification"], 
                      environ.val_data[task_key]['yc_aggr_weights'],
                      verbose = True)
 

258.0
[0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1.
 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0.
 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0.
 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 

In [28]:
# del all_tgs, all_tgs2
del con,con2

NameError: name 'con' is not defined

In [32]:
# del con
ttl = 0

# con = np.ndarray()
appd_df = []
for i in range(1,11):
    task_key = f"task{i}"
    print(i, task_key, ' shape: ', environ.val_data[task_key]['yc_aggr_weights'].shape,  'classifiaction:', environ.val_metrics[task_key]['classification'].shape)
    tmp_df = environ.val_metrics[task_key]['classification'].where(pd.isnull,1)
    print(tmp_df.sum(axis=0))
    
    if i == 1:
        con = np.copy(environ.val_data[task_key]['yc_aggr_weights'])
        all_tgs = environ.val_metrics[task_key]['classification'].copy()
        print("initialize", con.shape, all_tgs.shape)
    else:
        con = np.hstack((con, environ.val_data[task_key]['yc_aggr_weights']))
        all_tgs = all_tgs.append(environ.val_metrics[task_key]['classification'])
        print("concatenate: ",task_key, "    ", con.shape, all_tgs.shape)
        
    ttl += environ.val_data[task_key]['yc_aggr_weights'].shape[0]
    
print('ttl : ', ttl,  'con.shape:', con.shape, 'all_tgs.shape', all_tgs.shape)

1 task1  shape:  (472,) classifiaction: (472, 9)
roc_auc_score     293.0
auc_pr            293.0
avg_prec_score    293.0
f1_max            293.0
p_f1_max          293.0
kappa             293.0
kappa_max         293.0
p_kappa_max       293.0
bceloss           293.0
dtype: float64
initialize (472,) (472, 9)
2 task2  shape:  (624,) classifiaction: (624, 9)
roc_auc_score     386.0
auc_pr            386.0
avg_prec_score    386.0
f1_max            386.0
p_f1_max          386.0
kappa             386.0
kappa_max         386.0
p_kappa_max       386.0
bceloss           386.0
dtype: float64
concatenate:  task2      (1096,) (1096, 9)
3 task3  shape:  (688,) classifiaction: (688, 9)
roc_auc_score     610.0
auc_pr            610.0
avg_prec_score    610.0
f1_max            610.0
p_f1_max          610.0
kappa             610.0
kappa_max         610.0
p_kappa_max       610.0
bceloss           610.0
dtype: float64
concatenate:  task3      (1784,) (1784, 9)
4 task4  shape:  (192,) classifiaction: (192, 9

In [ ]:
all_tgs2 = pd.concat(environ.val_metrics[f"task{i}"]['classification'] for i in range(1,11))

all_tgs2.info()
all_tgs2.head(50)

In [185]:
con2 = np.hstack([ environ.val_data[f"task{i}"]['yc_aggr_weights'] for i in range(1,11)])
con2.shape

(3568,)

In [179]:
# all_tgs.index = range(all_tgs.shape[0])

In [180]:
# print(all_tgs2[-50:])

In [187]:
all_tgs2_mod = all_tgs2.where(pd.isnull, 1) * con2[:,None]
all_tgs2_mod.sum(axis = 0)

roc_auc_score     448.0
auc_pr            448.0
avg_prec_score    448.0
f1_max            448.0
p_f1_max          448.0
kappa             448.0
kappa_max         448.0
p_kappa_max       448.0
bceloss           448.0
dtype: float64

In [174]:
# con3 = pd.concat([environ.val_metrics['task1']['classification'],environ.val_metrics['task2']['classification'] ])
# print(con3)

In [188]:
tmp2 = aggregate_results(all_tgs2, con2, verbose = True)
 

 wsum: 1314.0   df.shape: (3568, 9)   df2: (3568, 9)  df2.sum(axis=0): 
 roc_auc_score     448.0
auc_pr            448.0
avg_prec_score    448.0
f1_max            448.0
p_f1_max          448.0
kappa             448.0
kappa_max         448.0
p_kappa_max       448.0
bceloss           448.0
dtype: float64

  DIVISOR 
-----------
roc_auc_score     0.002232
auc_pr            0.002232
avg_prec_score    0.002232
f1_max            0.002232
p_f1_max          0.002232
kappa             0.002232
kappa_max         0.002232
p_kappa_max       0.002232
bceloss           0.002232
dtype: float64

  DF 
------
      roc_auc_score    auc_pr  avg_prec_score    f1_max  p_f1_max  kappa  kappa_max  p_kappa_max   bceloss
task                                                                                                      
0               NaN       NaN             NaN       NaN       NaN    NaN        NaN          NaN       NaN
1          0.000000  0.250000        0.500000  0.666667  0.320208    0.0       

In [203]:
pp.pprint(environ.val_metrics['aggregated'])
print(environ.val_metrics['aggregated']['sc_loss'] )
print(environ.val_metrics['aggregated']["logloss"] ) 

{   'auc_pr': 0.6818710653441948,
    'avg_prec_score': 0.7442062851089334,
    'bceloss': 0.6360303774875189,
    'f1_max': 0.7990044449679128,
    'kappa': 0.08006046274015079,
    'kappa_max': 0.5820449674723311,
    'logloss': 0.00032448763622636064,
    'p_f1_max': 0.48441957962599447,
    'p_kappa_max': 0.532944236640885,
    'roc_auc_score': 0.6855984778924409,
    'sc_loss': 0.24956344102169398}
0.24956344102169398
0.00032448763622636064


In [201]:
pp.pprint(tmp2)
pp.pprint(tmp3)

roc_auc_score     0.685598
auc_pr            0.681871
avg_prec_score    0.744206
f1_max            0.799004
p_f1_max          0.484420
kappa             0.080060
kappa_max         0.582045
p_kappa_max       0.532944
bceloss           0.636030
dtype: float64


In [190]:
all_tasks_classification_metrics = []
all_tasks_aggregation_weights    = [] 

for i in range(1,11):
    task_key = f"task{i}"
    print(i, task_key, ' shape: ', environ.val_data[task_key]['yc_aggr_weights'].shape,  'classifiaction:', environ.val_metrics[task_key]['classification'].shape)
    tmp_df = environ.val_metrics[task_key]['classification'].where(pd.isnull,1)
    print(tmp_df.sum(axis=0))
    
    all_tasks_classification_metrics.append(environ.val_metrics[task_key]['classification'])
    all_tasks_aggregation_weights.append(environ.val_data[task_key]['yc_aggr_weights'])
            

1 task1  shape:  (472,) classifiaction: (472, 9)
roc_auc_score     60.0
auc_pr            60.0
avg_prec_score    60.0
f1_max            60.0
p_f1_max          60.0
kappa             60.0
kappa_max         60.0
p_kappa_max       60.0
bceloss           60.0
dtype: float64
2 task2  shape:  (624,) classifiaction: (624, 9)
roc_auc_score     46.0
auc_pr            46.0
avg_prec_score    46.0
f1_max            46.0
p_f1_max          46.0
kappa             46.0
kappa_max         46.0
p_kappa_max       46.0
bceloss           46.0
dtype: float64
3 task3  shape:  (688,) classifiaction: (688, 9)
roc_auc_score     126.0
auc_pr            126.0
avg_prec_score    126.0
f1_max            126.0
p_f1_max          126.0
kappa             126.0
kappa_max         126.0
p_kappa_max       126.0
bceloss           126.0
dtype: float64
4 task4  shape:  (192,) classifiaction: (192, 9)
roc_auc_score     29.0
auc_pr            29.0
avg_prec_score    29.0
f1_max            29.0
p_f1_max          29.0
kappa         

In [ ]:
all_tgs3 = pd.concat(all_tasks_classification_metrics)
con3 = np.concatenate(all_tasks_aggregation_weights)

In [191]:
all_tgs3.info()
all_tgs3.head(20)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3568 entries, 0 to 71
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   roc_auc_score   525 non-null    float64
 1   auc_pr          525 non-null    float64
 2   avg_prec_score  525 non-null    float64
 3   f1_max          525 non-null    float64
 4   p_f1_max        525 non-null    float32
 5   kappa           525 non-null    float64
 6   kappa_max       525 non-null    float64
 7   p_kappa_max     525 non-null    float32
 8   bceloss         525 non-null    float64
dtypes: float32(2), float64(7)
memory usage: 250.9 KB


,roc_auc_score,auc_pr,avg_prec_score,f1_max,p_f1_max,kappa,kappa_max,p_kappa_max,bceloss
task,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.000000,0.250000,0.500000,0.666667,0.320208,0.0,0.0,0.587646,0.780148
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.333333,0.166667,0.333333,0.500000,0.328739,0.0,0.2,0.328739,0.575516
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [200]:
tmp3 = aggregate_results( all_tgs3, con3, verbose = True)

 wsum: 1314.0   df.shape: (3568, 9)   df2: (3568, 9)  df2.sum(axis=0): 
 roc_auc_score     448.0
auc_pr            448.0
avg_prec_score    448.0
f1_max            448.0
p_f1_max          448.0
kappa             448.0
kappa_max         448.0
p_kappa_max       448.0
bceloss           448.0
dtype: float64

  DIVISOR 
-----------
roc_auc_score     0.002232
auc_pr            0.002232
avg_prec_score    0.002232
f1_max            0.002232
p_f1_max          0.002232
kappa             0.002232
kappa_max         0.002232
p_kappa_max       0.002232
bceloss           0.002232
dtype: float64

  DF 
------
      roc_auc_score    auc_pr  avg_prec_score    f1_max  p_f1_max  kappa  kappa_max  p_kappa_max   bceloss
task                                                                                                      
0               NaN       NaN             NaN       NaN       NaN    NaN        NaN          NaN       NaN
1          0.000000  0.250000        0.500000  0.666667  0.320208    0.0       

In [199]:
print(all_tgs2[0:1]['roc_auc_score'])
print(all_tgs3[0:1]['roc_auc_score'])
print((all_tgs2[0:1]['roc_auc_score'] == all_tgs3[0:1]['roc_auc_score']).all())
all_tgs2.compare(all_tgs3)

task
0   NaN
Name: roc_auc_score, dtype: float64
task
0   NaN
Name: roc_auc_score, dtype: float64
False


task


In [82]:
print(environ.val_data['task9']['yc_aggr_weights'].shape, con[3152:3496].shape)
print((environ.val_data['task9']['yc_aggr_weights'] == con[3152:3496]).all())a

(344,) (344,)
True


In [ ]:
print(f"Best Epoch :       {ns.best_epoch}\n"
      f"Best Iteration :   {ns.best_iter} \n"
      f"Best ROC AUC   :   {ns.best_roc_auc:.5f}\n"
      f"Best Precision :   {ns.best_accuracy:.5f}\n")
print()

In [50]:
print(f"Best Epoch :       {ns.best_epoch}\n"
      f"Best Iteration :   {ns.best_iter} \n"
      f"Best ROC AUC   :   {ns.best_roc_auc:.5f}\n"
      f"Best Precision :   {ns.best_accuracy:.5f}\n")
print()

Best Epoch :       3
Best Iteration :   7977 
Best ROC AUC   :   0.73690
Best Precision :   0.64175




In [22]:
print(f"Best Epoch :       {ns.best_epoch}\n"
      f"Best Iteration :   {ns.best_iter} \n"
      f"Best ROC AUC   :   {ns.best_roc_auc:.5f}\n"
      f"Best Precision :   {ns.best_accuracy:.5f}\n")
print()

Best Epoch :       1
Best Iteration :   2659 
Best ROC AUC   :   0.71991
Best Precision :   0.63031




In [24]:
print(f"Best Epoch :       {ns.best_epoch}\n"
      f"Best Iteration :   {ns.best_iter} \n"
      f"Best ROC AUC   :   {ns.best_roc_auc:.5f}\n"
      f"Best Precision :   {ns.best_accuracy:.5f}\n")
print()
for key in environ.val_metrics['aggregated']:
    print(f"{key:20s}    {environ.val_metrics['aggregated'][key]:0.4f}")
# pp.pprint(environ.val_metrics['aggregated'])

Best Epoch :       2
Best Iteration :   6258 
Best ROC AUC   :   0.79359
Best Precision :   0.72131


roc_auc_score           0.7936
auc_pr                  0.6974
avg_prec_score          0.7213
f1_max                  0.7534
p_f1_max                0.4367
kappa                   0.3055
kappa_max               0.5732
p_kappa_max             0.4947
bceloss                 0.4300
sc_loss                 0.0106
logloss                 0.0000


## Weight & Policy Training

### Weight/Policy Training Preparation

In [27]:
# environ.opt['is_curriculum'] = True
# environ.opt['curriculum_speed'] = 4
# ns.num_train_layers = None

In [26]:
ns.flag = 'update_weights'
model_initializations(ns, opt, environ, phase = ns.flag, policy_learning = True)
training_initializations(ns, opt, environ, dldrs, warmup = False)
# training_initializations(ns, opt, environ, dldrs, warmup_iterations = 200,  weight_iterations = 200, policy_iterations = 200, eval_iterations = 50, warmup = False)

 Model optimizers defined . . . policy_learning: True
 Model schedulers defined . . . policy_learning: True
 Metrics CSV file header written . . . 
 Model initializations complete . . . 
 training preparation: - check for CUDA - cuda available as device id: [0]
sparsechem_env.cuda()
 policy policy1 is None
 policy policy2 is None
 policy policy3 is None
 policy policy4 is None
 policy policy5 is None
 policy policy6 is None
 policy policy7 is None
 policy policy8 is None
 policy policy9 is None
 policy policy10 is None
 training preparation: - set print_freq to                                 : 1989 
 training preparation: - set number of batches per warmup training epoch to: 1989
 training preparation: - set number of batches per weight training epoch to: 1318
 training preparation: - set number of batches per policy training epoch to: 671
 training preparation: - set number of batches per validation to           : 675
 training preparation complete . . .


In [27]:
print( f" Backbone Initial LR            :      {environ.opt['train']['backbone_lr']:4f}      Current LR : {environ.optimizers['weights'].param_groups[0]['lr']} \n"
       f" Tasks    Initial LR            :      {environ.opt['train']['task_lr']:4f}      Current LR : {environ.optimizers['weights'].param_groups[1]['lr']}    \n"
       f" Policy   Initial LR            :      {environ.opt['train']['policy_lr']:4f}      Current LR : {environ.optimizers['alphas'].param_groups[0]['lr']}  \n")

 Backbone Initial LR            :      0.001000      Current LR : 0.001 
 Tasks    Initial LR            :      0.001000      Current LR : 0.001    
 Policy   Initial LR            :      0.001000      Current LR : 0.001  



In [28]:
print_heading( f"** {timestring()} \n"
               f"** Training epoch: {ns.current_epoch} iter: {ns.current_iter}   flag: {ns.flag} \n"
               f"** Set optimizer and scheduler to policy_learning = True (Switch weight optimizer from ADAM to SGD)\n"
               f"** Switch from Warm Up training to Alternate training Weights & Policy \n"
               f"** Take checkpoint and block gradient flow through Policy net", verbose=True)

------------------------------------------------------------------------------------------------------------------------
** 2022-08-18 19:12:57:984486 
** Training epoch: 10 iter: 19890   flag: update_weights 
** Set optimizer and scheduler to policy_learning = True (Switch weight optimizer from ADAM to SGD)
** Switch from Warm Up training to Alternate training Weights & Policy 
** Take checkpoint and block gradient flow through Policy net
------------------------------------------------------------------------------------------------------------------------ 



In [29]:
print(hasattr(environ, 'policy%d' % (1)))
print(getattr(environ, "policy1"))
print(environ.networks['mtl-net'].policys)

True
None
[]


In [30]:

print(environ.opt['train']['lambda_sparsity'])
print(environ.opt['train']['lambda_sharing'])
print(environ.opt['train']['decay_temp_freq'])

0.02
0.01
16


In [36]:

environ.opt['train']['lambda_sparsity'] = 0.001
environ.opt['train']['lambda_sharing']  = 0.01
environ.opt['train']['decay_temp_freq'] = 6
print(environ.opt['train']['lambda_sparsity'])
print(environ.opt['train']['lambda_sharing'])
print(environ.opt['train']['decay_temp_freq'])

0.001
0.01
6


In [37]:

print( f" ns.flag                        :      {ns.flag}")
print( f" num_train_layers               :      {ns.num_train_layers}")
print( f" environ.opt['is_curriculum']   :      {environ.opt['is_curriculum']}")
print( f" environ.opt['curriculum_speed']:      {environ.opt['curriculum_speed']}\n")
print( f" Backbone Initial LR            :      {environ.opt['train']['backbone_lr']:4f}      Current LR : {environ.optimizers['weights'].param_groups[0]['lr']} \n"
       f" Tasks    Initial LR            :      {environ.opt['train']['task_lr']:4f}      Current LR : {environ.optimizers['weights'].param_groups[1]['lr']}    \n"
       f" Policy   Initial LR            :      {environ.opt['train']['policy_lr']:4f}      Current LR : {environ.optimizers['alphas'].param_groups[0]['lr']}  \n")

print( f" Hard Sampling                  :      {environ.opt['train']['hard_sampling']}\n")

print( f" Sparsity regularization        :      {environ.opt['train']['lambda_sparsity']}\n"
       f" Sharing  regularization        :      {environ.opt['train']['lambda_sharing']} \n"
       f" Tasks    regularization        :      {environ.opt['train']['lambda_tasks']}   \n\n")

print( f" Gumbel Temp                    :      {environ.gumbel_temperature:.4f}         \n" #
       f" Gumbel Temp decay              :      {environ.opt['train']['decay_temp_freq']} \n") #

print( f" ns.current_epoch               :      {ns.current_epoch}")
print( f" ns.training_epochs             :      {ns.training_epochs} \n") 
print( f" ns.current_iters               :      {ns.current_iter}")  
print( f" Batches in warmup epoch        :      {ns.trn_iters_warmup}")
print( f" Batches in weight epoch        :      {ns.trn_iters_weights}")
print( f" Batches in policy epoch        :      {ns.trn_iters_policy}")
print( f" Batches in validation          :      {ns.eval_iters}")
print( f" num_train_layers               :      {ns.num_train_layers} \n")

 ns.flag                        :      update_weights
 num_train_layers               :      6
 environ.opt['is_curriculum']   :      False
 environ.opt['curriculum_speed']:      3

 Backbone Initial LR            :      0.001000      Current LR : 0.001 
 Tasks    Initial LR            :      0.001000      Current LR : 0.001    
 Policy   Initial LR            :      0.001000      Current LR : 0.001  

 Hard Sampling                  :      False

 Sparsity regularization        :      0.001
 Sharing  regularization        :      0.01 
 Tasks    regularization        :      1   


 Gumbel Temp                    :      4.0000         
 Gumbel Temp decay              :      6 

 ns.current_epoch               :      12
 ns.training_epochs             :      2 

 ns.current_iters               :      23868
 Batches in warmup epoch        :      1989
 Batches in weight epoch        :      1318
 Batches in policy epoch        :      671
 Batches in validation          :      675
 num_train

In [38]:
print_loss(ns.val_metrics, title = f"[e] Last ep:{ns.current_epoch}  it:{ns.current_iter} ")
print()
environ.display_trained_policy(ns.current_epoch)
environ.display_trained_logits(ns.current_epoch)
print_heading(f" Last Epoch Completed : {ns.current_epoch}       # of epochs to run:  {ns.training_epochs} -->  epochs {ns.current_epoch+1} to {ns.training_epochs + ns.current_epoch}"
              f"\n policy_learning rate : {environ.opt['train']['policy_lr']} "
              f"\n lambda_sparsity      : {environ.opt['train']['lambda_sparsity']}"
              f"\n lambda_sharing       : {environ.opt['train']['lambda_sharing']}"
              f"\n curriculum training  : {opt['is_curriculum']}     cirriculum speed: {opt['curriculum_speed']}     num_training_layers : {ns.num_train_layers}", 
              verbose = True)

[e] Last ep:12  it:23868  -  Total Loss: 2.2310     
Task: 2.2300   Sparsity: 1.04549e-03    Sharing: 1.00072e-05 


 ep:   12    softmax     s          softmax     s          softmax     s          softmax     s          softmax     s          softmax     s          softmax     s          softmax     s          softmax     s          softmax     s         
 ----- ----------------- -    ----------------- -    ----------------- -    ----------------- -    ----------------- -    ----------------- -    ----------------- -    ----------------- -    ----------------- -    ----------------- -    
  0    0.4538    0.5462  0    0.4652    0.5348  0    0.3973    0.6027  0    0.4549    0.5451  0    0.4204    0.5796  0    0.4631    0.5369  0    0.4475    0.5525  0    0.4489    0.5511  0    0.4307    0.5693  0    0.4526    0.5474  0
  1    0.4668    0.5332  0    0.4763    0.5237  0    0.4055    0.5945  0    0.4575    0.5425  0    0.4332    0.5668  0    0.4727    0.5273  0    0.4581    0.5419  0    

### Weight/Policy Training

In [39]:
weight_policy_training(ns, opt, environ, dldrs, display_policy = True, disable_tqdm = False, epochs = 2)

------------------------------------------------------------------------------------------------------------------------
 Last Epoch Completed : 12   # of epochs to run:  2 -->  epochs 13 to 14    
 policy_learning rate : 0.001      
 lambda_sparsity      : 0.001
 lambda_sharing       : 0.01 
 curriculum training  : False     cirriculum speed: 3     num_training_layers : 6
------------------------------------------------------------------------------------------------------------------------ 

 Ep  | Trunk LR  Heads LR  Polcy LR  Gmbl Tmp |  trn tsk    trn spar    trn shar   trn ttl |   logloss   bceloss  avg prec    aucroc     aucpr    f1_max |  val tsk    val spar    val shar     total | time |
  13 | 1.00e-03  1.00e-03  1.00e-03  4.00e+00 |   1.5934   8.383e-05   1.605e-05    1.5935 |   0.00000   0.47630   0.63846   0.72964   0.62506   0.67845 |   2.2253   5.227e-05   1.001e-05    2.2253 |1456.5 |
Previous best_epoch:    12   best iter: 23197   best_accuracy: 0.63875    best ROC auc

In [38]:
environ.display_trained_policy(ns.current_epoch)
environ.display_trained_logits(ns.current_epoch)



 ep:    8    softmax     s          softmax     s          softmax     s          softmax     s          softmax     s          softmax     s          softmax     s          softmax     s          softmax     s          softmax     s         
 ----- ----------------- -    ----------------- -    ----------------- -    ----------------- -    ----------------- -    ----------------- -    ----------------- -    ----------------- -    ----------------- -    ----------------- -    
  0    0.4589    0.5411  0    0.4410    0.5590  0    0.3997    0.6003  0    0.4436    0.5564  0    0.4145    0.5855  0    0.4546    0.5454  0    0.4417    0.5583  0    0.4384    0.5616  0    0.4293    0.5707  0    0.4569    0.5431  0
  1    0.4520    0.5480  0    0.4419    0.5581  0    0.4097    0.5903  0    0.4369    0.5631  0    0.4200    0.5800  0    0.4551    0.5449  0    0.4419    0.5581  0    0.4401    0.5599  0    0.4460    0.5540  0    0.4531    0.5469  0
  2    0.4522    0.5478  0    0.4316    0.5684  0 

### Close WandB run

In [39]:
ns.wandb_run.finish()

auc_pr,▁▄▄██▅█▆
avg_prec_score,▁▄▄██▅█▆
bceloss,█▅▄▂▁▂▁▁
best_accuracy,▁▄▄███
best_epoch,▁▂▃▅▆█
best_iter,▁▂▃▃▅█
best_roc_auc,▁▃▄███
epoch,▁▂▃▄▅▆▇█
f1_max,▁▃▄█▇▅█▅
gumbel_temp,▁▁▁▁▁▁▁▁
kappa,▁▃▅▇▇▇█▇


In [ ]:
# ns.best_epoch = 0
# from utils.notebook_modules import wrapup_phase
# wrapup_phase(ns, opt, environ)

In [ ]:
# environ.opt['train']['policy_lr']       = 0.002
# environ.opt['train']['lambda_sparsity'] = 0.05
# environ.opt['train']['lambda_sharing']  = 0.01
# environ.opt['train']['lambda_tasks']    = 1.0
# # environ.opt['train']['decay_temp_freq'] = 2

In [ ]:
print( f" Backbone Learning Rate      : {environ.opt['train']['backbone_lr']}\n"
       f" Tasks    Learning Rate      : {environ.opt['train']['task_lr']}\n"
       f" Policy   Learning Rate      : {environ.opt['train']['policy_lr']}\n")

print( f" Sparsity regularization     : {environ.opt['train']['lambda_sparsity']}\n"
       f" Sharing  regularization     : {environ.opt['train']['lambda_sharing']} \n\n"
       f" Tasks    regularization     : {environ.opt['train']['lambda_tasks']}   \n"
       f" Gumbel Temp                 : {environ.gumbel_temperature:.4f}         \n" 
       f" Gumbel Temp decay           : {environ.opt['train']['decay_temp_freq']}\n") 

print( f" current_iters               : {ns.current_iter}   \n"
       f" current_epochs              : {ns.current_epoch}  \n" 
       f" train_total_epochs          : {ns.training_epochs}\n" 
       f" stop_epoch_training         : {ns.stop_epoch_training}")

## Post Weight + Policy Training Stuff 

In [44]:
from torch import nn

In [46]:
print(isinstance(environ.networks['mtl-net'], nn.DataParallel))
print(environ.opt['backbone'])

False
SparseChem


In [49]:
for name, param in environ.networks['mtl-net'].named_parameters():
    print(f" {name:40s}  {param.shape} ")

 task1_logits                              torch.Size([6, 2]) 
 task2_logits                              torch.Size([6, 2]) 
 task3_logits                              torch.Size([6, 2]) 
 task4_logits                              torch.Size([6, 2]) 
 task5_logits                              torch.Size([6, 2]) 
 task6_logits                              torch.Size([6, 2]) 
 task7_logits                              torch.Size([6, 2]) 
 task8_logits                              torch.Size([6, 2]) 
 task9_logits                              torch.Size([6, 2]) 
 task10_logits                             torch.Size([6, 2]) 
 backbone.Input_Layer.linear.weight        torch.Size([32000, 4000]) 
 backbone.Input_Layer.linear.bias          torch.Size([4000]) 
 backbone.blocks.0.linear.weight           torch.Size([4000, 4000]) 
 backbone.blocks.0.linear.bias             torch.Size([4000]) 
 backbone.blocks.1.linear.weight           torch.Size([4000, 4000]) 
 backbone.blocks.1.linear.bias      

In [50]:
for name, param in environ.networks['mtl-net'].backbone.named_parameters():
        print(f" {name:40s}  {param.shape} ")

 Input_Layer.linear.weight                 torch.Size([32000, 4000]) 
 Input_Layer.linear.bias                   torch.Size([4000]) 
 blocks.0.linear.weight                    torch.Size([4000, 4000]) 
 blocks.0.linear.bias                      torch.Size([4000]) 
 blocks.1.linear.weight                    torch.Size([4000, 4000]) 
 blocks.1.linear.bias                      torch.Size([4000]) 
 blocks.2.linear.weight                    torch.Size([4000, 4000]) 
 blocks.2.linear.bias                      torch.Size([4000]) 
 blocks.3.linear.weight                    torch.Size([4000, 4000]) 
 blocks.3.linear.bias                      torch.Size([4000]) 
 blocks.4.linear.weight                    torch.Size([4000, 4000]) 
 blocks.4.linear.bias                      torch.Size([4000]) 
 blocks.5.linear.weight                    torch.Size([4000, 4000]) 
 blocks.5.linear.bias                      torch.Size([4000]) 


In [53]:
for name, param in environ.networks['mtl-net'].named_parameters():
    if 'task' in name and 'fc' in name:    
        print(f" {name:40s}  {param.shape} ")

 task1_fc1_c0.linear.weight                torch.Size([472, 4000]) 
 task1_fc1_c0.linear.bias                  torch.Size([472]) 
 task2_fc1_c0.linear.weight                torch.Size([624, 4000]) 
 task2_fc1_c0.linear.bias                  torch.Size([624]) 
 task3_fc1_c0.linear.weight                torch.Size([688, 4000]) 
 task3_fc1_c0.linear.bias                  torch.Size([688]) 
 task4_fc1_c0.linear.weight                torch.Size([192, 4000]) 
 task4_fc1_c0.linear.bias                  torch.Size([192]) 
 task5_fc1_c0.linear.weight                torch.Size([620, 4000]) 
 task5_fc1_c0.linear.bias                  torch.Size([620]) 
 task6_fc1_c0.linear.weight                torch.Size([184, 4000]) 
 task6_fc1_c0.linear.bias                  torch.Size([184]) 
 task7_fc1_c0.linear.weight                torch.Size([224, 4000]) 
 task7_fc1_c0.linear.bias                  torch.Size([224]) 
 task8_fc1_c0.linear.weight                torch.Size([148, 4000]) 
 task8_fc1_c0.linear.b

In [38]:
# pp.pprint(environ.losses)
# pp.pprint(environ.val_metrics)
# environ.num_layers, environ.networks['mtl-net'].num_layers
# environ.batch

In [ ]:
# pp.pprint(environ.val_metrics)

In [ ]:
# print_loss(environ.val_metrics, title = f"[Final] ep:{current_epoch}  it:{current_iter}",)
# environ.display_trained_policy(current_epoch)
# environ.display_trained_logits(current_epoch)
# environ.log_file.flush()

In [ ]:
environ.networks['mtl-net'].backbone.layer_config

In [ ]:
num_blocks = 6
num_policy_layers = 6
gt =  torch.ones((num_blocks)).long()
gt0 =  torch.zeros((num_blocks)).long()
print(gt)
print(gt0)

loss_weights = ((torch.arange(0, num_policy_layers, 1) + 1).float() / num_policy_layers)
print(loss_weights)

In [ ]:
if environ.opt['diff_sparsity_weights'] and not environ.opt['is_sharing']:
    print(' cond 1')
    ## Assign higher weights to higher layers 
    loss_weights = ((torch.arange(0, num_policy_layers, 1) + 1).float() / num_policy_layers)
    print(f"{task_key} sparsity error:  {2 * (loss_weights[-num_blocks:] * environ.cross_entropy2(logits[-num_blocks:], gt)).mean()})")
    print_dbg(f" loss_weights :  {loss_weights}", verbose = True)
    print_dbg(f" cross_entropy:  {environ.cross_entropy2(logits[-num_blocks:], gt)}  ", verbose = True)
    print_dbg(f" loss[sparsity][{task_key}]: {self.losses['sparsity'][task_key] } ", verbose = True)

else:
    print('\n cond 2')
    print_dbg(f"Compute CrossEntropyLoss between \n Logits   : \n{logits[-num_blocks:]} \n and gt: \n{gt} \n", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[-num_blocks:], gt)}")
    
    print('\n cond 2')
    print_dbg(f"Compute CrossEntropyLoss between Logits      : {logits[-1:]}  and gt: {gt[-1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[-1:], gt[-1:])} \n")
    print_dbg(f"Compute CrossEntropyLoss between Logits      : {logits[-1:]}  and gt: {gt0[-1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[-1:], gt0[-1:])} \n")
    
    print('\n cond 3')    
    print_dbg(f"Compute CrossEntropyLoss between Logits   : {logits[0:1]}  and gt: {gt[0:1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[0:1], gt[0:1])} \n")
    print_dbg(f"Compute CrossEntropyLoss between Logits   : {logits[0:1]}  and gt: {gt0[0:1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[0:1], gt0[0:1])} \n")
        
        

In [ ]:
# flag = 'update_w'
# environ.fix_alpha
# environ.free_w(opt['fix_BN'])

flag = 'update_alpha'
environ.fix_weights()
environ.free_alpha()

In [ ]:
environ.networks['mtl-net'].num_layers

In [ ]:
print(f"current_iters         : {current_iter}")  
print(f"current_epochs           : {current_epoch}") 
print(f"train_total_epochs    : {train_total_epochs}") 

train_total_epochs += 5

print(f"current_iters         : {current_iter}")  
print(f"current_epochs           : {current_epoch}") 
print(f"train_total_epochs    : {train_total_epochs}") 

## Check values

In [ ]:
print(" ns.check_for_improvment_wait:  {ns.check_for_improvment_wait}")
print(" ns.curriculum_epochs:          {ns.curriculum_epochs}")

In [ ]:
# pp.pprint(environ.val_metrics)
df = environ.val_metrics['task1']['classification']

In [ ]:
print(df[pd.notna(df.roc_auc_score)])

In [ ]:
df[pd.notna(df.roc_auc_score)].mean()

In [ ]:
# environ.display_trained_policy(ns.current_epoch,out=[sys.stdout])
# environ.num_tasks
# print(environ.get_policy_prob().shape)
# print(environ.val_data['task1'].keys())
# print(environ.val_data['task1']['yc_ind'][0][:40])
# print(environ.val_data['task1']['yc_ind'][1][:40])
# print(environ.val_data['task1']['yc_data'][:40])
# print(environ.val_data['task1']['yc_hat'][:40])
# environ.display_trained_policy(ns.current_epoch,out=[sys.stdout])
# environ.display_trained_logits(ns.current_epoch,out=[sys.stdout])
batch = next(dldrs.warmup_trn_loader)   

In [ ]:
batch.keys()

In [ ]:
print(batch['x_ind'].shape)
print(batch['x_ind'][:,:20])

In [ ]:
print(type(batch['x_data']))
print(batch['x_data'].shape)

In [ ]:
print(len(batch['row_id']))
print(batch['row_id'])

In [ ]:
print(batch['batch_size'])

In [ ]:
# pp.pprint(environ.val_metrics['task1'])
pp.pprint(environ.val_metrics)
# pp.pprint(ns.val_metrics)
# print((environ.val_data['task1']['yc_data']).sum())
# print(len(environ.val_data['task1']['yc_ind'][1]))
# print(len(environ.val_data['task1']['yc_data']))
# print(len(environ.val_data['task1']['yc_hat']))

In [ ]:
type(ns.val_metrics['aggregated'])

In [ ]:
pp.pprint(ns.trn_losses)

In [ ]:
(environ.val_data['task1']['yc_data'][0] == environ.val_data['task1']['yc_data']).all()

In [ ]:
from utils.sparsechem_utils import compute_metrics, aggregate_results
import pandas
cc = compute_metrics(cols   = environ.val_data['task1']['yc_ind'][1], 
                     y_true = environ.val_data['task1']['yc_data'], 
                     y_score= environ.val_data['task1']['yc_hat'] ,
                     num_tasks=100)


In [ ]:
 df   = pd.DataFrame({"task"   : environ.val_data['task1']['yc_ind'][1], 
                      "y_true" : environ.val_data['task1']['yc_data'],  
                      "y_score": environ.val_data['task1']['yc_hat']})

In [ ]:
for task, frame in df.groupby("task", sort=True):
    print(f" task {task}")
    print(frame.head(10))

In [ ]:
# df
df.groupby("task", sort=True).count()

In [ ]:
pp.pprint(environ.val_metrics)

In [ ]:
print(environ.batch_data['task1']['yc_aggr_weights'])
environ.batch['task1']['aggr_weights']

In [ ]:
c2 = aggregate_results(cc)

In [ ]:
dldrs.trainset0.tasks_weights_list

In [ ]:

# environ.opt['train']['policy_lr'] = 0.01
# opt['train']['policy_lr']         = 0.01
# environ.opt['train']['lambda_sparsity'] = 0.1
# environ.opt['train']['lambda_sharing']  = 0.01
# environ.opt['train']['lambda_tasks']    = 1.0
# environ.opt['train']['decay_temp_freq'] = 2
# print(environ.optimizers['alphas'].param_groups)
# print(environ.optimizers['weights'].param_groups)
# print('current lr: ', environ.optimizers['alphas'].param_groups[0]['lr'],)
# print('current lr: ', environ.optimizers['weights'].param_groups[0]['lr'])
# print('current lr: ', environ.optimizers['weights'].param_groups[1]['lr'])

## Post Warm-up Training stuff

In [ ]:
pp.pprint(environ.val_metrics)

In [ ]:
environ.networks['mtl-net'].arch_parameters()

In [ ]:
p = environ.get_sample_policy(hard_sampling = False)
print(p)
p = environ.get_policy_prob()
print(p)
p = environ.get_policy_logits()
print(p)

# p = environ.get_current_policy()
# print(p)

In [ ]:
a = softmax([0.0, 1])
print(a)
sampled = np.random.choice((1, 0), p=a)
print(sampled)

In [ ]:
print(environ.optimizers['weights'])
print(environ.schedulers['weights'].get_last_lr())

In [ ]:
print('losses.keys      : ', environ.losses.keys())
print('losses[task]keys : ', environ.losses['task1'].keys())
pp.pprint(environ.losses)

In [ ]:
print( environ.val_metrics.keys())
# pp.pprint(val_metrics)
print(type(environ.val_metrics['aggregated']))
print()
print(type(environ.val_metrics['task1']['classification_agg']))
print()
pp.pprint(environ.val_metrics)

In [ ]:
# import pickle
# with open("val_metrics.pkl", mode= 'wb') as f:
#         pickle.dump(val_metrics, f)
    
# with open('val_metrics.pkl', 'rb') as f:    
#     tst_val_metrics = pickle.load(f)

In [ ]:
# print(environ.input.shape) 
# a = getattr(environ, 'task1_pred')
# yc_data = environ.batch['task1_data']
# print(yc_data.shape)
# yc_ind = environ.batch['task1_ind']
# print(yc_ind.shape)
# yc_hat_all = getattr(environ, 'task1_pred')
# print(yc_hat_all.shape)
# yc_hat  = yc_hat_all[yc_ind[0], yc_ind[1]]
# print(yc_hat_all.shape, yc_hat.shape)

# 
# environ.losses
# loss = {}
# for key in environ.losses.keys():
#     loss[key] = {}
#     for subkey, v in environ.losses[key].items():
#         print(f" key:  {key}   subkey: {subkey} ")
#         if isinstance(v, torch.Tensor):
#             loss[key][subkey] = v.data
#             print(f" Tensor  -  key:  {key}   subkey: {subkey}           value type: {type(v)}  value: {v:.4f}")
#         else:
#             loss[key][subkey] = v
#             print(f" integer -  key:  {key}   subkey: {subkey}           value type: {type(v)}  value: {v:.4f}")
# pp.pprint(tst_val_metrics)             

In [ ]:
# print('metrics.keys: ', environ.metrics.keys())
# print('metrics[task].keys: ', environ.metrics['task1'].keys())
# pp.pprint(environ.metrics['task1'])
# pp.pprint(environ.losses['task1']['total'])

In [ ]:
# title='Iteration'
# for t_id, _ in enumerate(environ.tasks):
#     task_key = f"task{t_id+1}"
# #     print_heading(f"{title}  {current_iter}  {task_key} : {val_metrics[task_key]['classification_agg']}", verbose = True)

#     for key, _  in val_metrics[task_key]['classification_agg'].items():
#         print('%s/%-20s'%(task_key, key), val_metrics[task_key]['classification_agg'][key], current_iter)
#         print(f"{task_key:s}/{key:20s}", val_metrics[task_key]['classification_agg'][key], current_iter)
#         print()
#             # print_current_errors(os.path.join(self.log_dir, 'loss.txt'), current_iter,key, loss[key], time.time() - start_time)

In [ ]:
# environ.print_loss(current_iter, start_time, metrics = val_metrics['loss'], verbose=True)
# print(opt['lambdas'])
# p = (opt['lambdas'][0] * environ.losses['tasks']['task1'])
# print(p)

# environ.print_val_metrics(current_iter, start_time, val_metrics , title='validation', verbose=True)    

In [ ]:
# print(current_iter)
# print_metrics_cr(current_iter, t1 - t0, None, val_metrics , True)
# environ.print_val_metrics(current_iter, start_time, val_metrics, title='validation', verbose = True)

In [ ]:
print(f" val_metric keys               : {val_metrics.keys()}")
print(f" loss keys                     : {val_metrics['loss'].keys()}")
print(f" task1 keys                    : {val_metrics['task1'].keys()}")
print(f" task1 classification keys     : {val_metrics['task1']['classification'].keys()}")
print(f" task1 classification_agg keys : {val_metrics['task1']['classification_agg'].keys()}")
print()
print(f" task1                       : {val_metrics['task1']['classification_agg']['loss']:5f}")
print(f" task2                       : {val_metrics['task2']['classification_agg']['loss']:5f}")
print(f" task3                       : {val_metrics['task3']['classification_agg']['loss']:5f}")
print(f" loss                        : {val_metrics['loss']['total']:5f}")
print(f" train_time                  : {val_metrics['train_time']:2f}")
print(f" epoch                       : {val_metrics['epoch']}")


## Losses and Metrics

In [ ]:
trn_losses = environ.losses

In [ ]:
print_metrics_cr(current_epoch, time.time() - start_time, trn_losses, environ.val_metrics , num_prints)      

In [ ]:
# print_metrics_cr(current_epoch, time.time() - start_time, trn_losses, environ.val_metrics , num_prints)      

In [ ]:
# pp.pprint(environ.losses)
pp.pprint(trn_losses)

In [ ]:
pp.pprint(environ.val_metrics)

In [ ]:
# environ.opt['train']['Lambda_sharing'] = 0.5
# opt['train']['Lambda_sharing'] = 0.5

# environ.opt['train']['policy_lr'] = 0.001
# opt['train']['policy_lr'] = 0.001

In [ ]:
environ.losses.keys()
pp.pprint(environ.losses)

In [ ]:
tmp = environ.get_loss_dict()
print(tmp.keys())
pp.pprint(tmp)

In [ ]:
print(opt['diff_sparsity_weights'])
print(opt['is_sharing'])
print(opt['diff_sparsity_weights'] and not opt['is_sharing'])
print(environ.opt['train']['Lambda_sharing'])
print(opt['train']['Lambda_sharing'])
print(environ.opt['train']['Lambda_sparsity'])
print(opt['train']['Lambda_sparsity'])
print(environ.opt['train']['policy_lr'])
print(opt['train']['policy_lr'])

## Policy / Logit stuff

In [ ]:
from scipy.special          import softmax

In [ ]:
np.set_printoptions(precision=8,edgeitems=3, infstr='inf', linewidth=150, nanstr='nan')
torch.set_printoptions(precision=8,linewidth=132)

#### `get_task_logits(n)` Get logits for task group n

In [ ]:
task_logits = environ.get_task_logits(1)
print(task_logits)

#### `get_arch_parameters()`: Get last used logits from network

In [ ]:
import torch.optim as optim
arch_parameters      = environ.get_arch_parameters()
print(arch_parameters)

In [ ]:
import torch.optim as optim
arch_parameters      = environ.get_arch_parameters()
print(arch_parameters)

#### `get_policy_logits()`:  Get Policy Logits - returns same as `get_arch_parameters()`

In [ ]:
logs = environ.get_policy_logits()
for i in logs:
    print(i, '\n')
# probs = softmax(logs, axis= -1)
# for i in probs:
#     print(i, '\n')

#### `get_policy_prob()` : Gets the softmax of the logits

In [ ]:
policy_softmaxs = environ.get_policy_prob()
for i in policy_softmaxs:
    print(i, '\n')

#### `get_sample_policy( hard_sampling = False)` : Calls test_sample_policy of network with random choices based on softmax of logits

In [ ]:
policy_softmaxs = environ.get_policy_prob()
policies,logits = environ.get_sample_policy(hard_sampling = False)

for l, p, s in zip(logits, policies, policy_softmaxs) :
    for  l_row, p_row, s_row in zip(l, p, s):
        print( l_row,'\t', p_row, '\t', s_row)
    print('\n')

#### `get_sample_policy( hard_sampling = True)` : Calls test_sample_policy of network using ARGMAX of logits

In [ ]:
policy_softmaxs = environ.get_policy_prob()
hard_policies, logits = environ.get_sample_policy(hard_sampling = True)

for p,l,s in zip(hard_policies, logits, policy_softmaxs) :
    for  p_row, l_row, s_row in zip(p, l, s):
        print( l_row,'\t', p_row, '\t', s_row)
    print('\n')

#### Print

In [ ]:
print(f" Layer    task 1      task 2      task 3")
print(f" -----    ------      ------      ------")
for idx, (l1, l2, l3) in enumerate(zip(hard_policies[0], hard_policies[1], hard_policies[2]),1):
    print(f"   {idx}      {l1}       {l2}       {l3}")
    

    print(f"\n\n where [p1  p2]:  p1: layer is selected    p2: layer is not selected")

In [ ]:
def display_trained_policy(iter):

    policy_softmaxs = environ.get_policy_prob()
    policy_argmaxs = 1-np.argmax(policy_softmaxs, axis = -1)
    print(f"  Trained polcies at iteration: {iter} ")
    print(f"                   task 1                           task 2                         task 3        ")
    print(f" Layer       softmax        select          softmax        select          softmax        select   ")
    print(f" -----    ---------------   ------       ---------------   ------       ---------------   ------   ")
    for idx, (l1,l2,l3,  p1,p2,p3) in enumerate(zip(policy_softmaxs[0], policy_softmaxs[1], policy_softmaxs[2], policy_argmaxs[0], policy_argmaxs[1], policy_argmaxs[2]),1):
        print(f"   {idx}      {l1[0]:.4f}   {l1[1]:.4f}   {p1:4d}    {l2[0]:11.4f}   {l2[1]:.4f}   {p2:4d}    {l3[0]:11.4f}   {l3[1]:.4f}   {p3:4d}")

    print()
# print(f"\n\n where [p1  p2]:  p1: layer is selected    p2: layer is not selected")

In [ ]:
display_trained_policy(5)

In [ ]:
print(f"                        POLICIES (SOFTMAX)                                       task 3          ")
print(f" Layer    task1              task2            task3 softmax         softmax         argmax         softmax         argmax   ")
print(f" -----    -------------     -------------     -------------   ------   ")
for idx, (l1,l2,l3, h1,h2,h3) in enumerate(zip(policy_softmaxs[0], policy_softmaxs[1], policy_softmaxs[2],hard_policies[0], hard_policies[1], hard_policies[2]),1):
    print(f"   {idx}      {l1[0]:.4f} {l1[1]:.4f}     {l2[0]:.4f} {l2[1]:.4f}     {l3[0]:.4f} {l3[1]:.4f}    {h3}")
    
print(f"\n\n where [p1  p2]:  p1: layer is selected    p2: layer is not selected")

In [ ]:
# print(policy_softmaxs[2], np.argmax(1-policy_softmaxs[2], axis = -1))
print(policy_softmaxs, np.argmax(policy_softmaxs, axis = -1))

#### `get_current_logits()` : Calls test_sample_policy of network using ARGMAX of logits

In [ ]:
logits  = (environ.get_current_logits())
for i in logits:
    print(i ,'\n')

#### `get_current_policy()` : Calls test_sample_policy of network using ARGMAX of logits

In [ ]:
pols  = (environ.get_current_policy())

for i in pols:
    print(i ,'\n')

#### `gumbel_softmax()`  

In [ ]:
np.set_printoptions(precision=8,edgeitems=3, infstr='inf', linewidth=150, nanstr='nan', floatmode = 'maxprec_equal')
torch.set_printoptions(precision=8,linewidth=132)

In [ ]:
print(environ.temp)
# tau = environ.temp
tau = 1
for i in range(3): 
    logits_tensor = torch.tensor(logits[0])
    # Sample soft categorical using reparametrization trick:
    gumbel_soft = F.gumbel_softmax(logits_tensor, tau=tau, hard=False).cpu().numpy() 

    # Sample hard categorical using "Straight-through" trick:
    gumbel_hard  = F.gumbel_softmax(logits_tensor, tau=tau, hard=True).cpu().numpy()
    
    for l, gs, gh in zip(lgts, gumbel_soft, gumbel_hard):
        print(f"   {l}   \t {gs}            \t {gh}")
#     print(lgts)
#     print(gumbel_soft)
#     print(gumbel_hard)
    print()

In [ ]:
for lgts in logits:
    logits_tensor = torch.tensor(lgts)
    print(lgts)
    # Sample soft categorical using reparametrization trick:
    gumbel_soft = F.gumbel_softmax(logits_tensor, tau=1, hard=False)
    print(gumbel_soft)

    # Sample hard categorical using "Straight-through" trick:
    gumbel_hard  = F.gumbel_softmax(logits_tensor, tau=1, hard=True)
    print(gumbel_hard)
    print()

In [ ]:
smax = scipy.special.softmax(logs, axis =1)
# smax = np.array( 
# [[0.46973792, 0.530262  ],
#  [0.45025694, 0.549743  ],
#  [0.4443086 , 0.5556915 ],
#  [0.4138397 , 0.58616036],
#  [0.4140113 , 0.5859887 ],
#  [0.42114905, 0.57885087]])

print(smax.shape)
print(smax)
print(smax[0])
print(smax[0].sum())
print(np.random.choice((1,0), p =smax[0]))

In [ ]:
logs = np.array(
[[0.33064184, 0.42053092],
 [0.3532089 , 0.52056104],
 [0.3888512 , 0.5680909 ],
 [0.42039296, 0.694217  ],
 [0.4519742 , 0.73311865],
 [0.48401102, 0.7522658 ]],
)